# LangGraph 完全学習ガイド - 実践ノートブック

---

## このノートブックについて

このノートブックは、LangGraphを使った**議事録要約AI**を一から構築するハンズオン教材です。

### 対象者
- Pythonの基礎がわかる方
- LangChainの基本を学んだ方
- AIエージェント開発に興味がある方

### 学習の流れ
1. **環境準備** - 必要なライブラリの確認
2. **State定義** - データ構造の設計
3. **Node定義** - 処理関数の実装
4. **ルーティング** - 条件分岐の実装
5. **グラフ組み立て** - 全体の接続
6. **実行テスト** - 動作確認

### 重要な注意事項
- Vertex AI APIがなくても、**モックモード**で構文と動作を確認できます
- 各セルを**上から順番に**実行してください
- わからない部分は、次のセルに進む前に**コメントを読み直して**ください

---

# STEP 1: 環境準備

## 1.1 ライブラリのインストール

LangGraphを使うには、以下のライブラリが必要です：

| ライブラリ | 役割 | 必須？ |
|-----------|------|--------|
| `langgraph` | グラフ構造のワークフロー構築 | ✅ 必須 |
| `langchain-google-vertexai` | Gemini APIへの接続 | ⚠️ API使用時のみ |
| `langchain-core` | プロンプトテンプレートなど | ⚠️ API使用時のみ |

In [ ]:
# ============================================
# ライブラリのインストール（必要な場合のみ実行）
# ============================================
# 
# 以下のコメントを外して実行すると、必要なライブラリがインストールされます。
# すでにインストール済みの場合は、このセルはスキップしてOKです。
#
# !pip install langgraph langchain-google-vertexai

## 1.2 基本ライブラリの詳細解説

以下では、各ライブラリの役割と使い方を初心者向けに解説します。

---

### 📚 typing モジュール（Python標準ライブラリ）

Pythonに「型」の概念を追加するためのモジュールです。

#### TypedDict
辞書（dict）に**フィールド名と型を定義**できる機能です。

```python
# 🔴 普通のdict（型がない）
# → タイポしても気づかない
# → IDEの補完が効かない
data = {"name": "田中", "age": 30}
data["nmae"]  # タイポ！でもエラーにならない

# 🟢 TypedDict（型がある）
# → タイポするとIDEが警告
# → 補完が効いて開発が快適
class Person(TypedDict):
    name: str
    age: int
```

#### Literal
**特定の文字列だけ**を許容する型です。

```python
# Literal["A", "B"] = "A"か"B"以外は許可しない
def get_mode() -> Literal["detailed", "simple"]:
    return "detailed"  # OK
    return "detaled"   # NG（タイポ！IDEが警告）
```

#### Annotated
型に**追加情報（メタデータ）**を付けるための機能です。

```python
# Annotated[型, メタデータ] の形式
# LangGraphでは「reducer」という追加情報を付ける
logs: Annotated[List[str], add]
# ↑「List[str]型で、マージ時にはaddを使う」という意味
```

#### List
リスト型を示す型ヒントです。

```python
# List[str] = 文字列のリスト
topics: List[str]  # ["議題1", "議題2", ...]

# List[int] = 整数のリスト
numbers: List[int]  # [1, 2, 3, ...]
```

---

### 📚 operator モジュール（Python標準ライブラリ）

Pythonの演算子を「関数」として提供するモジュールです。

#### add 関数
`+` 演算子と同じ動作をする関数です。

```python
from operator import add

# 数値の場合
add(1, 2)  # → 3  （1 + 2と同じ）

# リストの場合（連結になる）
add([1, 2], [3, 4])  # → [1, 2, 3, 4]

# LangGraphでの使い方
# logs: Annotated[List[str], add]
# → ノードが返した logs は「上書き」ではなく「追加」される
```

**なぜ `+` ではなく `add` を使う？**
- `Annotated` には「関数」を渡す必要がある
- `+` は演算子なので渡せない
- `add` は関数なので渡せる

---

### 📚 datetime モジュール（Python標準ライブラリ）

日付と時刻を扱うモジュールです。

```python
from datetime import datetime

# 現在の日時を取得
now = datetime.now()

# フォーマットして文字列に変換
formatted = now.strftime('%Y-%m-%d %H:%M')
# → "2024-01-15 10:30"

# フォーマット記号の意味
# %Y = 年（4桁）  例: 2024
# %m = 月（2桁）  例: 01
# %d = 日（2桁）  例: 15
# %H = 時（24時間） 例: 10
# %M = 分（2桁）  例: 30
```

In [ ]:
# ============================================
# 基本ライブラリのインポート
# ============================================
#
# 【各ライブラリの役割】
# - TypedDict: 型付き辞書を作成（Stateの定義に使用）
# - Literal: 特定の文字列だけを許容する型（条件分岐の戻り値に使用）
# - Annotated: 型に追加情報を付ける（reducerに使用）
# - List: リスト型のヒント
# - add: リストを連結するためのreducer関数
#
from typing import TypedDict, Literal, Annotated, List
from operator import add
from datetime import datetime

print("✅ 基本ライブラリのインポート成功！")
print("")
print("📦 インポートしたもの:")
print("   - TypedDict: 型付き辞書の作成")
print("   - Literal: 特定の値のみ許容する型")
print("   - Annotated: 型にメタ情報を追加")
print("   - List: リスト型")
print("   - add: リスト連結用のreducer")
print("   - datetime: 日時処理")

## 1.3 LangGraph ライブラリの詳細解説

### LangGraphとは？

LangChain社が開発した**ワークフロー構築フレームワーク**です。
複数の処理（Node）を**グラフ構造**で接続し、複雑なAIエージェントを作れます。

---

### 📚 langgraph.graph モジュール

#### StateGraph クラス
ワークフロー全体を定義するクラスです。

```python
from langgraph.graph import StateGraph

# StateGraphを作成（Stateの型を指定）
workflow = StateGraph(MeetingState)

# ノードを追加
workflow.add_node("node_name", node_function)

# エッジを追加（ノード間の接続）
workflow.add_edge("node_a", "node_b")

# 条件分岐を追加
workflow.add_conditional_edges("node_a", condition_func, mapping)

# コンパイル（実行可能にする）
graph = workflow.compile()
```

#### START と END（特殊なノード）
処理の開始点と終了点を表す定数です。

```python
from langgraph.graph import START, END

# 処理の開始
workflow.add_edge(START, "first_node")  # 最初に first_node が実行される

# 処理の終了
workflow.add_edge("last_node", END)  # last_node の後に処理が終わる
```

---

### 主要メソッドの詳細

#### add_node(name, function)
ノード（処理関数）をグラフに追加します。

| パラメータ | 説明 |
|-----------|------|
| `name` | ノードの識別子（エッジ接続時に使用） |
| `function` | 実行する関数（Stateを受け取り、dictを返す） |

```python
workflow.add_node("summarize", summarize_node)
# ↑ "summarize" という名前で summarize_node関数を登録
```

#### add_edge(from_node, to_node)
2つのノード間に**固定の接続**を追加します。

```python
workflow.add_edge("summarize", "check_length")
# ↑ summarize の後に必ず check_length が実行される
```

#### add_conditional_edges(from_node, condition_func, mapping)
**条件分岐**を追加します。

| パラメータ | 説明 |
|-----------|------|
| `from_node` | 分岐元のノード名 |
| `condition_func` | 条件を判定する関数（Stateを受け取り、文字列を返す） |
| `mapping` | `{戻り値: 行き先ノード}` の辞書 |

```python
workflow.add_conditional_edges(
    "check_length",     # ここから分岐
    route_by_length,    # この関数で行き先を決定
    {
        "detailed": "extract_topics",  # "detailed"が返されたらここへ
        "simple": "format_doc",        # "simple"が返されたらここへ
    }
)
```

#### compile()
グラフを**実行可能な状態**に変換します。

```python
graph = workflow.compile()
# ↑ 以降は graph.invoke() で実行できる
```

---

### グラフの実行

#### invoke(initial_state)
グラフを実行し、最終状態を返します。

```python
result = graph.invoke({
    "raw_text": "会議テキスト...",
    "logs": []
})
# result には全フィールドが埋まった最終状態が入る
print(result["final_doc"])  # 生成された議事録
```

In [ ]:
# ============================================
# LangGraph のインポート
# ============================================
#
# 【インポート内容】
# - StateGraph: グラフを作るクラス。「会社」に相当
# - START: 処理の開始点。「入社初日」のイメージ
# - END: 処理の終了点。「退社」のイメージ
#
try:
    from langgraph.graph import StateGraph, START, END
    print("✅ LangGraph のインポート成功！")
    print("")
    print("📦 インポートしたもの:")
    print("   - StateGraph: ワークフロー（グラフ）を作成するクラス")
    print("   - START: 処理の開始点を表す特殊な定数")
    print("   - END: 処理の終了点を表す特殊な定数")
    LANGGRAPH_AVAILABLE = True
except ImportError as e:
    print(f"❌ LangGraph のインポート失敗: {e}")
    print("")
    print("🔧 解決方法:")
    print("   上のセルで以下のコマンドを実行してください:")
    print("   !pip install langgraph")
    LANGGRAPH_AVAILABLE = False

## 1.4 LangChain Vertex AI の詳細解説（オプション）

### Vertex AI とは？
Google Cloudの**AI/MLプラットフォーム**です。
Gemini（GoogleのLLM）やその他の機械学習モデルを使用できます。

---

### 📚 langchain-google-vertexai パッケージ

#### ChatVertexAI クラス
Vertex AIのチャットモデル（Gemini等）を呼び出すためのクラスです。

```python
from langchain_google_vertexai import ChatVertexAI

# LLMを初期化
llm = ChatVertexAI(
    model="gemini-1.5-flash",  # 使用するモデル
    temperature=0              # 出力のランダム性（0=安定, 1=創造的）
)

# 実行
response = llm.invoke("こんにちは")
print(response.content)  # AIの応答テキスト
```

| パラメータ | 説明 | おすすめ値 |
|-----------|------|-----------|
| `model` | 使用するモデルのID | `"gemini-1.5-flash"`（高速・低コスト） |
| `temperature` | 出力のランダム性 | `0`（議事録は安定した出力が望ましい） |

**モデルの選び方:**
- `gemini-1.5-flash` - 高速・低コスト（推奨）
- `gemini-1.5-pro` - 高精度（複雑なタスク向け）

---

### 📚 langchain-core パッケージ

#### ChatPromptTemplate クラス
**プロンプト（AIへの指示）**を構造化して作成するクラスです。

```python
from langchain_core.prompts import ChatPromptTemplate

# プロンプトを作成
prompt = ChatPromptTemplate.from_messages([
    ("system", "あなたは〇〇のプロです。"),  # AIへの役割設定
    ("human", "{text}")                       # ユーザーからの入力
])

# {text} はプレースホルダ
# invoke時に実際の値が埋め込まれる
```

**メッセージの種類:**

| 種類 | 説明 | 使い方 |
|------|------|--------|
| `system` | AIへの役割設定・指示 | 「あなたは〇〇のプロです」 |
| `human` | ユーザーからの入力 | 実際のテキストを渡す |
| `ai` | AIの過去の応答（会話履歴用） | マルチターン会話で使用 |

---

### LangChainの「チェーン」パターン

`|` 演算子で処理を**パイプラインのように**繋げます。

```python
# prompt | llm = 「プロンプトを作成 → LLMに渡す」という処理の流れ
chain = prompt | llm

# 実行
result = chain.invoke({"text": "会議テキスト..."})
print(result.content)  # AIの応答
```

**なぜチェーンを使う？**
- 処理の流れが視覚的にわかりやすい
- 再利用・テストがしやすい
- LangChainの他のコンポーネントと組み合わせやすい

In [ ]:
# ============================================
# LangChain Vertex AI のインポート（オプション）
# ============================================
#
# 【注意】
# Vertex AI APIがない環境でも、モックモードで学習できます。
# このセルが失敗しても、次に進めます。
#
try:
    from langchain_google_vertexai import ChatVertexAI
    from langchain_core.prompts import ChatPromptTemplate
    print("✅ LangChain Vertex AI のインポート成功！")
    print("")
    print("📦 インポートしたもの:")
    print("   - ChatVertexAI: Gemini APIを呼び出すクラス")
    print("   - ChatPromptTemplate: プロンプトを作成するクラス")
    print("")
    print("💡 実際にAPIを使う場合は、Google Cloud認証が必要です:")
    print("   gcloud auth application-default login")
    VERTEXAI_AVAILABLE = True
except ImportError as e:
    print(f"⚠️ LangChain Vertex AI のインポート失敗: {e}")
    print("")
    print("📝 これは問題ありません！")
    print("   → モックモードで学習を続けられます")
    print("   → 構文とロジックの学習には影響しません")
    VERTEXAI_AVAILABLE = False

## 1.5 モックモードの設定

### モックモードとは？
実際のLLMを呼び出さずに、**ダミーの応答**を返すモードです。

**使いどころ**:
- APIキーがない環境での学習
- 開発中のテスト
- APIコストを節約したいとき

**設定方法**:
- `USE_MOCK = True`: モックを使う（API呼び出しなし）
- `USE_MOCK = False`: 実際のAPIを使う

In [ ]:
# ============================================
# モックモードの設定
# ============================================
#
# 【設定値の意味】
# - True: モックを使う（API呼び出しなし、学習向け）
# - False: 実際のVertex AI APIを使う（本番向け）
#
# 【変更方法】
# API環境がある場合は、以下を False に変更してください
#
USE_MOCK = True  # ← ここを変更

if USE_MOCK:
    print("🔧 モックモード: ON")
    print("")
    print("📝 説明:")
    print("   - LLM呼び出しの代わりにダミー応答を使用します")
    print("   - APIキーなしでも学習を進められます")
    print("   - 構文とロジックの確認が目的です")
else:
    if VERTEXAI_AVAILABLE:
        print("🚀 本番モード: ON")
        print("")
        print("📝 説明:")
        print("   - 実際のVertex AI (Gemini) APIを使用します")
        print("   - Google Cloud認証が必要です")
        print("   - API呼び出しごとに少額の課金が発生します")
    else:
        print("❌ エラー: Vertex AIライブラリがインストールされていません")
        print("   USE_MOCK = True に設定してください")
        USE_MOCK = True

---

# STEP 2: State（状態）の定義

## 2.1 Stateとは何か？

### 会社に例えると...

LangGraphを**会社**に例えると、Stateは**回覧ファイル（クリップボード）**です。

```
┌─────────────────────────────────────────────────────────────────┐
│  🏢 議事録要約株式会社                                          │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  📋 回覧ファイル（State）                                       │
│  ┌─────────────────────────────────────────────────────────┐   │
│  │ 会議テキスト: _____________（最初に記入済み）           │   │
│  │ 要約欄:       _____________（田中さんが記入）           │   │
│  │ 議題欄:       _____________（鈴木さんが記入）           │   │
│  │ 最終議事録:   _____________（佐藤さんが記入）           │   │
│  └─────────────────────────────────────────────────────────┘   │
│                                                                 │
│  👤 社員（Node）たちが順番に自分の欄を埋めていく                │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

### ポイント
1. **最初に「入力データ」だけが埋まっている**
2. **各Node（社員）が自分の担当欄を埋めて、次に回す**
3. **最後には全欄が埋まった「完成版」になる**

## 2.2 MeetingStateの設計

今回作る議事録要約AIのStateは、以下のフィールドを持ちます：

### フィールド一覧

| フィールド | 型 | 役割 | 誰が設定？ |
|-----------|-----|------|------------|
| `raw_text` | str | 生の会議テキスト | ユーザー（最初に） |
| `summary` | str | 要約テキスト | summarize_node |
| `is_long` | bool | 長い会議フラグ | check_length_node |
| `topics` | List[str] | 議題リスト | extract_topics_node |
| `action_items` | List[str] | アクション一覧 | extract_actions_node |
| `final_doc` | str | 最終議事録 | format_doc_node |
| `logs` | List[str] | 処理ログ | 全Node |

In [ ]:
# ============================================
# MeetingState（議事録要約AIの状態）を定義
# ============================================
#
# 【TypedDictの使い方】
# class クラス名(TypedDict):
#     フィールド名: 型
#
# 【Annotatedとは？】
# Annotated[型, reducer] の形式で、
# 「この型には特別なマージルールがある」ことを示します。
#
# 【addとは？】
# operator.add は「+」演算子と同じ動作をする関数です。
# リストに対して使うと、リストを連結します。
# 例: add([1, 2], [3, 4]) → [1, 2, 3, 4]
#

class MeetingState(TypedDict):
    """
    議事録要約AIの状態（State）
    
    このクラスは「回覧ファイル」のようなもの。
    各Node（社員）が順番に自分の担当欄を埋めていきます。
    
    【設計のポイント】
    1. 入力: 最初に与えるデータ（raw_text）
    2. 中間: 処理途中で埋まるデータ（summary, is_long）
    3. 出力: 最終的に取り出すデータ（topics, action_items, final_doc）
    4. ログ: デバッグ用の処理記録（logs）
    """
    
    # ============================
    # 📥 入力データ（ユーザーが与える）
    # ============================
    # 会議の生テキスト
    # 例: Zoomの文字起こし、手書きメモなど
    raw_text: str
    
    # ============================
    # 🔄 中間データ（Nodeが埋めていく）
    # ============================
    # 要約結果（300文字程度）
    # summarize_nodeが設定する
    summary: str
    
    # 長い会議かどうかのフラグ
    # check_length_nodeが設定する
    # 条件分岐の判定に使用される
    is_long: bool
    
    # 議題リスト
    # extract_topics_nodeが設定する
    # 例: ["予算について", "スケジュール確認", "次回日程"]
    topics: List[str]
    
    # アクションアイテム（ToDo）のリスト
    # extract_actions_nodeが設定する
    # 例: ["田中: 資料作成（来週火曜まで）", "鈴木: レビュー実施"]
    action_items: List[str]
    
    # ============================
    # 📤 出力データ（最終成果物）
    # ============================
    # 整形された議事録（Markdown形式）
    # format_doc_nodeが設定する
    final_doc: str
    
    # ============================
    # 📝 処理ログ（デバッグ用）
    # ============================
    # Annotated[List[str], add] の意味:
    #   - List[str] 型である
    #   - add をreducerとして使う
    #   - つまり、新しい値は「上書き」ではなく「追加」される
    #
    # 例: logs = ["A"] の状態で return {"logs": ["B"]} すると
    #     logs = ["A", "B"] になる（上書きではない！）
    logs: Annotated[List[str], add]


# 定義の確認
print("✅ MeetingState の定義成功！")
print("")
print("📋 定義されたフィールド:")
for field_name, field_type in MeetingState.__annotations__.items():
    print(f"   - {field_name}: {field_type}")

## 2.3 Stateの動きを確認してみよう

Stateがどのように変化するか、簡単な例で確認しましょう。

In [ ]:
# ============================================
# Stateの動きを確認する実験
# ============================================
#
# 【実験内容】
# 1. 初期状態のStateを作成
# 2. Nodeが返す「部分更新」をマージ
# 3. 変化を確認
#

# 初期状態（ユーザーが最初に渡すデータ）
initial_state = {
    "raw_text": "田中: 来週の締め切りについて確認。",
    "logs": []  # 空のリスト
}

print("📋 初期状態:")
print(f"   raw_text: {initial_state['raw_text']}")
print(f"   logs: {initial_state['logs']}")
print("")

# Nodeが返す「部分更新」（summarize_nodeの戻り値をイメージ）
node_output = {
    "summary": "来週の締め切りを確認する短い会議",
    "logs": ["[summarize] 要約完了"]  # この1件が追加される
}

print("📝 Nodeの出力（部分更新）:")
print(f"   summary: {node_output['summary']}")
print(f"   logs: {node_output['logs']}")
print("")

# LangGraphが自動で行うマージ処理をシミュレート
# 通常のフィールド: 上書き
# Annotated[..., add]のフィールド: 追加
merged_state = {
    **initial_state,
    "summary": node_output["summary"],  # 上書き
    "logs": initial_state["logs"] + node_output["logs"]  # 追加（addの効果）
}

print("✅ マージ後の状態:")
print(f"   raw_text: {merged_state['raw_text']}（変更なし）")
print(f"   summary: {merged_state['summary']}（新規追加）")
print(f"   logs: {merged_state['logs']}（追加された）")

---

# STEP 3: Node（処理関数）の定義

## 3.1 Nodeとは何か？

### 会社に例えると...

Nodeは**社員（担当者）**です。

```
👤 要約担当の田中さん（summarize_node）
   - 回覧ファイル（State）を受け取る
   - 「raw_text」を読む
   - 「summary」欄に記入する
   - 回覧ファイルを次の人に渡す
```

### Nodeの基本形

```python
def node_name(state: MeetingState) -> dict:
    """
    Stateを受け取り、更新内容をdictで返す
    """
    # 1. Stateから必要な値を読む
    value = state["some_field"]
    
    # 2. 何かしらの処理をする
    result = do_something(value)
    
    # 3. 更新したいフィールドだけをdictで返す
    return {
        "updated_field": result,
        "logs": ["処理完了"]
    }
```

## 3.2 要約ノード（summarize_node）

### このNodeの役割
- **入力**: 生の会議テキスト（`raw_text`）
- **処理**: LLMで300文字程度に要約
- **出力**: 要約テキスト（`summary`）

### なぜ要約から始める？
長い会議テキストをそのまま後続処理に渡すと：
- トークン数が増えてコストが上がる
- 情報が多すぎてLLMの精度が下がる

**→ まず要約して、情報を圧縮する！**

In [ ]:
# ============================================
# 要約ノード（summarize_node）
# ============================================
#
# 【役割】
# 会議テキスト → 300文字程度の要約
#
# 【引数】
# state: MeetingState - 現在の状態（回覧ファイル）
#
# 【戻り値】
# dict - 更新したいフィールドだけを含む辞書
#        {"summary": "...", "logs": [...]} の形式
#

def summarize_node(state: MeetingState) -> dict:
    """
    【要約ノード】
    会議テキスト → 300文字程度の要約
    
    Parameters:
        state: 現在の状態（raw_textを使用）
    
    Returns:
        dict: {"summary": 要約テキスト, "logs": ["処理ログ"]}
    """
    
    if USE_MOCK:
        # ================================
        # モックモード（API呼び出しなし）
        # ================================
        # テスト用に、先頭100文字 + "..." を要約として使用
        raw_text = state["raw_text"]
        summary = raw_text[:100] + "...（モック要約）"
        print(f"[summarize_node] モックモード: 先頭100文字を使用")
        
    else:
        # ================================
        # 本番モード（実際のLLM呼び出し）
        # ================================
        # LLMを初期化
        llm = ChatVertexAI(
            model="gemini-1.5-flash",  # 高速・低コスト
            temperature=0              # 創造性を抑えて安定した出力
        )
        
        # プロンプトを定義
        # system: AIへの指示
        # human: ユーザーからの入力（{text}はプレースホルダ）
        prompt = ChatPromptTemplate.from_messages([
            ("system", """あなたは議事録作成のプロです。
以下の会議内容を300文字程度で要約してください。
- 重要な決定事項を優先
- 具体的な数字や期日は必ず含める
- 箇条書きではなく文章で"""),
            ("human", "{text}")
        ])
        
        # LangChainの「チェーン」を作成
        # prompt | llm = プロンプトを作って → LLMに渡す
        chain = prompt | llm
        
        # 実行（Stateから生テキストを取り出して渡す）
        result = chain.invoke({"text": state["raw_text"]})
        summary = result.content
        print(f"[summarize_node] LLM呼び出し完了")
    
    # ================================
    # 戻り値（部分更新）
    # ================================
    # 更新したいフィールドだけを返す
    # LangGraphが自動でStateにマージしてくれる
    return {
        "summary": summary,
        "logs": ["[summarize] 要約完了"]
    }


print("✅ summarize_node 定義完了")
print("")
print("📝 この関数の特徴:")
print("   - State の raw_text を読む")
print("   - summary フィールドを更新する")
print("   - logs にログを追加する")

## 3.3 長さチェックノード（check_length_node）

### このNodeの役割
- **入力**: 生の会議テキスト（`raw_text`）
- **処理**: テキストの長さをチェック
- **出力**: 長い会議フラグ（`is_long`）

### ポイント
**LLMを使わない処理もNodeにできる！**

Nodeは「関数」なので、LLM呼び出し以外の処理も自由にできます。
ここでは単純なPythonの文字列長チェックを行います。

In [ ]:
# ============================================
# 長さチェックノード（check_length_node）
# ============================================
#
# 【役割】
# 会議テキストの長さで「詳細モード or 簡易モード」を判定
#
# 【ポイント】
# - LLMを使わない純粋なPython処理
# - 条件分岐の判定に使うフラグを設定
#

def check_length_node(state: MeetingState) -> dict:
    """
    【長さチェックノード】
    会議テキストの長さで処理モードを判定
    
    判定基準:
        - 2000文字以上 → 詳細モード（is_long = True）
        - 2000文字未満 → 簡易モード（is_long = False）
    
    Parameters:
        state: 現在の状態（raw_textを使用）
    
    Returns:
        dict: {"is_long": bool, "logs": ["処理ログ"]}
    """
    
    # テキストの長さを取得
    text_length = len(state["raw_text"])
    
    # 2000文字以上なら「長い会議」と判定
    # この閾値は用途に応じて調整可能
    THRESHOLD = 2000
    is_long = text_length > THRESHOLD
    
    # デバッグ用の出力
    mode = "詳細モード" if is_long else "簡易モード"
    print(f"[check_length_node] テキスト長: {text_length}文字 → {mode}")
    
    # 戻り値
    return {
        "is_long": is_long,
        "logs": [f"[check_length] {text_length}文字 → {mode}"]
    }


print("✅ check_length_node 定義完了")
print("")
print("📝 この関数の特徴:")
print("   - LLMを使わない純粋なPython処理")
print("   - is_long フラグを設定（条件分岐に使用）")
print("   - 閾値は 2000文字")

## 3.4 議題抽出ノード（extract_topics_node）

### このNodeの役割
- **入力**: 要約テキスト（`summary`）
- **処理**: LLMで議題を抽出
- **出力**: 議題リスト（`topics`）

### 実装のポイント
LLMの出力を「改行区切り」にして、`split("\n")`で分割します。

**なぜJSON形式にしない？**
- LLMはJSONを正確に出力できないことがある
- 改行区切りの方がシンプルで確実

In [ ]:
# ============================================
# 議題抽出ノード（extract_topics_node）
# ============================================
#
# 【役割】
# 要約から主要な議題を抽出（リスト形式）
#
# 【入力】
# state["summary"] - 前のNodeで生成された要約
#
# 【出力】
# {"topics": ["議題1", "議題2", ...], "logs": [...]}
#

def extract_topics_node(state: MeetingState) -> dict:
    """
    【議題抽出ノード】
    要約から主要な議題を抽出（リスト形式）
    
    実装のポイント:
        - LLMに「1行1議題」で出力させる
        - split("\n") でリストに変換
        - JSON形式より確実に動作する
    
    Parameters:
        state: 現在の状態（summaryを使用）
    
    Returns:
        dict: {"topics": List[str], "logs": ["処理ログ"]}
    """
    
    if USE_MOCK:
        # ================================
        # モックモード
        # ================================
        # テスト用のダミー議題
        topics = [
            "プロジェクト進捗の確認",
            "予算に関する議論",
            "次回ミーティングの日程調整"
        ]
        print(f"[extract_topics_node] モックモード: ダミー議題を返却")
        
    else:
        # ================================
        # 本番モード
        # ================================
        llm = ChatVertexAI(model="gemini-1.5-flash", temperature=0)
        
        # プロンプト: 1行1議題、番号なし、最大5個
        prompt = ChatPromptTemplate.from_messages([
            ("system", """以下の会議要約から、議題を抽出してください。
- 1行に1議題
- 議題のみを出力（番号や記号なし）
- 最大5個まで"""),
            ("human", "{summary}")
        ])
        
        chain = prompt | llm
        result = chain.invoke({"summary": state["summary"]})
        
        # 改行で分割してリスト化
        # strip() で前後の空白を削除
        # 空行は除外（if line.strip()）
        topics = [
            line.strip()
            for line in result.content.split("\n")
            if line.strip()
        ]
        print(f"[extract_topics_node] LLM呼び出し完了: {len(topics)}件抽出")
    
    return {
        "topics": topics,
        "logs": [f"[topics] {len(topics)}件抽出"]
    }


print("✅ extract_topics_node 定義完了")
print("")
print("📝 この関数の特徴:")
print("   - summary を読んで topics を設定")
print("   - LLMの出力を改行で分割してリスト化")
print("   - 長い会議（詳細モード）でのみ実行される")

## 3.5 アクションアイテム抽出ノード（extract_actions_node）

### このNodeの役割
- **入力**: 要約テキスト（`summary`）
- **処理**: LLMでアクションアイテムを抽出
- **出力**: アクションリスト（`action_items`）

### アクションアイテムとは？
「誰が」「何を」「いつまでに」やるのかを明確にしたタスクのこと。

例:
- 田中: 資料作成（来週火曜まで）
- 鈴木: レビュー実施（今週金曜まで）

In [ ]:
# ============================================
# アクションアイテム抽出ノード（extract_actions_node）
# ============================================
#
# 【役割】
# 「誰が」「何を」「いつまでに」を抽出
#
# 【出力フォーマット】
# "担当者: タスク内容（期限）"
#

def extract_actions_node(state: MeetingState) -> dict:
    """
    【アクションアイテム抽出ノード】
    「誰が」「何を」「いつまでに」を抽出
    
    出力フォーマット:
        "担当者: タスク内容（期限があれば記載）"
    
    Parameters:
        state: 現在の状態（summaryを使用）
    
    Returns:
        dict: {"action_items": List[str], "logs": ["処理ログ"]}
    """
    
    if USE_MOCK:
        # ================================
        # モックモード
        # ================================
        actions = [
            "田中: 資料作成（来週火曜まで）",
            "鈴木: レビュー実施（今週金曜まで）",
            "山田: テスト環境準備（来週月曜まで）"
        ]
        print(f"[extract_actions_node] モックモード: ダミーアクションを返却")
        
    else:
        # ================================
        # 本番モード
        # ================================
        llm = ChatVertexAI(model="gemini-1.5-flash", temperature=0)
        
        prompt = ChatPromptTemplate.from_messages([
            ("system", """以下の会議要約から、アクションアイテムを抽出してください。
形式: 担当者: タスク内容（期限があれば記載）
- 1行に1アクション
- 具体的なタスクのみ抽出（曖昧なものは除外）"""),
            ("human", "{summary}")
        ])
        
        chain = prompt | llm
        result = chain.invoke({"summary": state["summary"]})
        
        # 改行で分割してリスト化
        actions = [
            line.strip()
            for line in result.content.split("\n")
            if line.strip()
        ]
        print(f"[extract_actions_node] LLM呼び出し完了: {len(actions)}件抽出")
    
    return {
        "action_items": actions,
        "logs": [f"[actions] {len(actions)}件抽出"]
    }


print("✅ extract_actions_node 定義完了")
print("")
print("📝 この関数の特徴:")
print("   - summary を読んで action_items を設定")
print("   - 担当者・タスク・期限を抽出")
print("   - 長い会議（詳細モード）でのみ実行される")

## 3.6 整形ノード（format_doc_node）

### このNodeの役割
- **入力**: 全ての中間データ（summary, topics, action_items）
- **処理**: Markdown形式に整形
- **出力**: 最終議事録（`final_doc`）

### ポイント
**LLMを使わない！**

テンプレートに値を埋め込むだけなので、LLM呼び出しは不要です。
- コスト削減
- 高速化
- 出力が安定

---

### 📚 使用する特殊な関数・構文

#### dict.get(key, default) メソッド
辞書から**安全に**値を取得するメソッドです。

```python
state = {"name": "田中"}

# 🔴 普通のアクセス（キーがないとエラー）
state["age"]  # KeyError!

# 🟢 get()を使う（キーがなくてもエラーにならない）
state.get("age", 0)  # → 0（デフォルト値が返る）
state.get("name", "不明")  # → "田中"（値がある場合はその値）
```

**なぜget()を使う？**
- 簡易モードでは `topics` や `action_items` が設定されない
- get()を使えばエラーなく空リストを取得できる

#### f-string（フォーマット文字列）
文字列の中に**変数を埋め込む**構文です。

```python
name = "田中"
age = 30

# 🔴 従来の方法（読みにくい）
message = "名前: " + name + ", 年齢: " + str(age)

# 🟢 f-stringを使う（読みやすい！）
message = f"名前: {name}, 年齢: {age}"
# → "名前: 田中, 年齢: 30"
```

#### 三重クォート（複数行文字列）
`"""..."""` で**複数行の文字列**を作成できます。

```python
doc = """# 議事録
## 要約
ここに要約が入ります。

## 議題
- 議題1
- 議題2
"""
```

#### リスト内包表記
リストを**1行で変換**する構文です。

```python
topics = ["予算", "スケジュール", "次回日程"]

# 🔴 従来の方法（長い）
result = []
for t in topics:
    result.append(f"- {t}")

# 🟢 リスト内包表記（1行でスッキリ）
result = [f"- {t}" for t in topics]
# → ["- 予算", "- スケジュール", "- 次回日程"]
```

#### str.join() メソッド
リストを**1つの文字列に結合**するメソッドです。

```python
items = ["- 予算", "- スケジュール", "- 次回日程"]

# 改行で結合
result = "\n".join(items)
# → "- 予算\n- スケジュール\n- 次回日程"
```

#### or 演算子（短絡評価）
**空文字列やNoneの場合にデフォルト値**を使う技法です。

```python
topics_md = ""  # 空文字列

# or の左が「偽」（空文字列、None、0、[]など）なら右を返す
result = topics_md or "(議題なし)"
# → "(議題なし)"

topics_md = "- 予算"  # 値がある場合
result = topics_md or "(議題なし)"
# → "- 予算"
```

In [ ]:
# ============================================
# 整形ノード（format_doc_node）
# ============================================
#
# 【役割】
# 全ての情報をMarkdown形式の議事録に整形
#
# 【特徴】
# - LLMを使わない（テンプレート処理のみ）
# - コスト削減＆高速化
# - state.get() で安全に値を取得（キーがなくてもエラーにならない）
#

def format_doc_node(state: MeetingState) -> dict:
    """
    【整形ノード】
    全ての情報をMarkdown形式の議事録に整形
    
    LLMを使わない処理:
        テンプレートに値を埋め込むだけ
        → コスト削減＆高速化
    
    Parameters:
        state: 現在の状態（summary, topics, action_itemsを使用）
    
    Returns:
        dict: {"final_doc": str, "logs": ["処理ログ"]}
    """
    
    # ================================
    # 議題をMarkdownリストに変換
    # ================================
    # state.get("topics", []) の意味:
    #   - "topics" キーがあれば、その値を返す
    #   - なければ、空のリスト [] を返す
    # これにより、簡易モードでtopicsがなくてもエラーにならない
    topics = state.get("topics", [])
    topics_md = "\n".join([f"- {t}" for t in topics])
    
    # ================================
    # アクションアイテムをMarkdownリストに変換
    # ================================
    # "- [ ]" はMarkdownのチェックボックス形式
    actions = state.get("action_items", [])
    actions_md = "\n".join([f"- [ ] {a}" for a in actions])
    
    # ================================
    # Markdown形式で議事録を生成
    # ================================
    # f-string（f"..."）を使って、変数を埋め込む
    doc = f"""# 議事録
**作成日時**: {datetime.now().strftime('%Y-%m-%d %H:%M')}

## 要約
{state.get('summary', '(要約なし)')}

## 議題
{topics_md or '(議題なし)'}

## アクションアイテム
{actions_md or '(アクションなし)'}
"""
    
    print("[format_doc_node] 議事録生成完了")
    
    return {
        "final_doc": doc,
        "logs": ["[format] 議事録生成完了"]
    }


print("✅ format_doc_node 定義完了")
print("")
print("📝 この関数の特徴:")
print("   - LLMを使わない（テンプレート処理のみ）")
print("   - 全ての中間データを final_doc に統合")
print("   - state.get() で安全にデータを取得")

---

# STEP 4: ルーティング（条件分岐）の定義

## 4.1 ルーティングとは？

### 会社に例えると...

ルーティングは**業務フローの分岐**です。

```
📋 回覧ファイルが check_length_node に届いた
   ↓
   「この会議は長い？短い？」をチェック
   ↓
   ┌─────────────────────────────────────┐
   │ 長い（2000文字以上）               │
   │ → 詳細チームへ回覧                 │
   │   （議題抽出 → アクション抽出）     │
   └─────────────────────────────────────┘
   ┌─────────────────────────────────────┐
   │ 短い（2000文字未満）               │
   │ → 整形担当へ直行                   │
   │   （議題・アクション抽出をスキップ）│
   └─────────────────────────────────────┘
```

### ルーティング関数のルール
1. **Stateを読むだけ**（更新しない）
2. **文字列を返す**（次に進むNodeの名前）
3. **Literal型で安全に**（返せる値を限定）

In [ ]:
# ============================================
# ルーティング関数（route_by_length）
# ============================================
#
# 【役割】
# Stateの `is_long` フラグを見て、次の処理を決定する
#
# 【戻り値の型】
# Literal["detailed", "simple"] は
# 「この2つの文字列しか返さない」ことを示す
# → タイポ防止＆IDEの補完が効く
#
# 【通常のNodeとの違い】
# - Nodeは dict を返す（Stateを更新）
# - ルーティング関数は str を返す（次のNode名）
#

def route_by_length(state: MeetingState) -> Literal["detailed", "simple"]:
    """
    【長さによるルーティング】
    
    Stateの `is_long` フラグを見て、次の処理を決定する。
    
    ルーティングの規則:
        - is_long = True → "detailed"（詳細モード）
        - is_long = False → "simple"（簡易モード）
    
    Parameters:
        state: 現在の状態（is_longを使用）
    
    Returns:
        Literal["detailed", "simple"]: 次に進むNodeの識別子
    
    Note:
        この関数はStateを「読むだけ」で「更新しない」点が
        通常のNodeと異なる。
    """
    
    # check_length_nodeで設定されたフラグを参照
    # state.get() で安全に取得（デフォルトはFalse）
    is_long = state.get("is_long", False)
    
    if is_long:
        # 長い会議 → 詳細な議題・アクション抽出へ
        print("[routing] 詳細モードへ分岐")
        return "detailed"
    else:
        # 短い会議 → シンプルな整形へ直行
        print("[routing] 簡易モードへ分岐")
        return "simple"


print("✅ route_by_length 定義完了")
print("")
print("📝 この関数の特徴:")
print("   - Stateを読むだけ（更新しない）")
print("   - 戻り値は文字列（次のNode名）")
print("   - Literal型で返せる値を限定")
print("")
print("🔀 分岐ルール:")
print("   - 'detailed' → extract_topics_node へ")
print("   - 'simple' → format_doc_node へ")

---

# STEP 5: グラフの組み立て

## 5.1 グラフ組み立ての流れ

### 手順
1. **StateGraphを作成** - Stateの型を指定
2. **Nodeを追加** - `add_node("名前", 関数)` で登録
3. **エッジを追加** - `add_edge("A", "B")` でA→B接続
4. **条件分岐を追加** - `add_conditional_edges()` で分岐
5. **compile()** - 実行可能なグラフに変換

### 図解

```
START
  │
  ▼
summarize ─────────────────────────────────────────
  │                                                │
  ▼                                                │
check_length                                       │
  │                                                │
  ├── is_long=True ──▶ extract_topics             │
  │                        │                       │
  │                        ▼                       │
  │                   extract_actions             │
  │                        │                       │
  │                        ▼                       │
  └── is_long=False ─────────────▶ format_doc ◀──┘
                                       │
                                       ▼
                                      END
```

In [ ]:
# ============================================
# グラフ作成関数（create_meeting_graph）
# ============================================
#
# 【この関数の役割】
# 全てのパーツ（Node、Edge、条件分岐）を組み立てて
# 実行可能なグラフを作成する
#

def create_meeting_graph():
    """
    議事録要約グラフを構築する
    
    グラフの流れ:
        START → summarize → check_length → [分岐]
            → (長い) extract_topics → extract_actions → format_doc → END
            → (短い) format_doc → END
    
    Returns:
        CompiledGraph: 実行可能なグラフオブジェクト
    """
    
    # ============================================
    # 1. グラフを作成（Stateの型を指定）
    # ============================================
    # StateGraph(型) で、その型に従ってデータが流れるグラフを作成
    # これは「会社を設立する」イメージ
    workflow = StateGraph(MeetingState)
    print("1. StateGraph 作成完了")
    
    # ============================================
    # 2. ノードを追加
    # ============================================
    # add_node("表示名", 関数)
    # 表示名はエッジ接続時に使う識別子
    # これは「社員を採用する」イメージ
    workflow.add_node("summarize", summarize_node)
    workflow.add_node("check_length", check_length_node)
    workflow.add_node("extract_topics", extract_topics_node)
    workflow.add_node("extract_actions", extract_actions_node)
    workflow.add_node("format_doc", format_doc_node)
    print("2. ノード追加完了（5個）")
    
    # ============================================
    # 3. エッジを追加（固定の接続）
    # ============================================
    # add_edge("A", "B") で A → B の接続を作る
    # START と END は特殊なNode（処理の開始/終了点）
    # これは「業務フローを定める」イメージ
    workflow.add_edge(START, "summarize")         # 開始 → 要約
    workflow.add_edge("summarize", "check_length")  # 要約 → 長さチェック
    print("3. 固定エッジ追加完了")
    
    # ============================================
    # 4. 条件分岐を追加
    # ============================================
    # add_conditional_edges(分岐元, 条件関数, マッピング)
    # - 分岐元: どのNodeから分岐するか
    # - 条件関数: 戻り値で行き先を決める関数
    # - マッピング: {戻り値: 行き先Node} の辞書
    workflow.add_conditional_edges(
        "check_length",     # どのNodeから分岐するか
        route_by_length,    # 条件を判定する関数
        {
            # 条件関数の戻り値 → 次に進むNode
            "detailed": "extract_topics",  # 長い → 議題抽出へ
            "simple": "format_doc",       # 短い → 整形へ直行
        }
    )
    print("4. 条件分岐追加完了")
    
    # ============================================
    # 5. 詳細モードの続きのエッジ
    # ============================================
    # 議題抽出 → アクション抽出 → 整形
    workflow.add_edge("extract_topics", "extract_actions")
    workflow.add_edge("extract_actions", "format_doc")
    print("5. 詳細モードのエッジ追加完了")
    
    # ============================================
    # 6. 終了エッジ
    # ============================================
    # 最後は全てENDへ（format_doc → END）
    workflow.add_edge("format_doc", END)
    print("6. 終了エッジ追加完了")
    
    # ============================================
    # 7. compile()で実行可能なグラフに変換
    # ============================================
    # compile() を呼ぶと、グラフが「確定」して実行可能になる
    # これは「会社を開業する」イメージ
    compiled_graph = workflow.compile()
    print("7. グラフのコンパイル完了")
    
    return compiled_graph


print("✅ create_meeting_graph 定義完了")

## 5.2 グラフを作成してみる

In [ ]:
# ============================================
# グラフの作成
# ============================================

print("=" * 50)
print("グラフを作成します...")
print("=" * 50)
print("")

try:
    graph = create_meeting_graph()
    print("")
    print("✅ グラフ作成成功！")
    print("")
    print("📊 作成されたグラフの構造:")
    print("   START → summarize → check_length → [分岐]")
    print("       → (詳細) extract_topics → extract_actions → format_doc → END")
    print("       → (簡易) format_doc → END")
except Exception as e:
    print(f"❌ グラフ作成失敗: {e}")
    print("")
    print("🔧 エラーの原因を確認してください:")
    print("   - LangGraphがインストールされているか")
    print("   - 各関数が正しく定義されているか")

---

# STEP 6: 実行テスト

## 6.1 テストの準備

2パターンのテストを行います：

1. **短い会議テスト** - 簡易モードが動作するか確認
2. **長い会議テスト** - 詳細モードが動作するか確認

## 6.2 短い会議テスト（簡易モード）

In [ ]:
# ============================================
# 短い会議テスト（簡易モード）
# ============================================
#
# 【期待される動作】
# START → summarize → check_length → format_doc → END
# （extract_topics, extract_actions はスキップ）
#

short_meeting = "田中: 来週の締め切りについて確認。鈴木: 火曜までに完了します。"

print("=" * 60)
print("【短い会議テスト（簡易モード）】")
print("=" * 60)
print(f"テキスト長: {len(short_meeting)}文字（2000文字未満）")
print("期待される分岐: 簡易モード（議題・アクション抽出をスキップ）")
print("")
print("-" * 60)
print("処理開始...")
print("-" * 60)

try:
    # graph.invoke() でグラフを実行
    # 引数は初期State（raw_textとlogsを設定）
    result = graph.invoke({
        "raw_text": short_meeting,
        "logs": []  # 空のリストで初期化（Annotated[..., add]なので必須）
    })
    
    print("")
    print("=" * 60)
    print("【生成された議事録】")
    print("=" * 60)
    print(result["final_doc"])
    
    print("=" * 60)
    print("【処理ログ】")
    print("=" * 60)
    for log in result["logs"]:
        print(f"  {log}")
    
    print("")
    print("✅ テスト成功！")
    
except Exception as e:
    print(f"❌ 実行エラー: {e}")
    import traceback
    traceback.print_exc()

## 6.3 長い会議テスト（詳細モード）

In [ ]:
# ============================================
# 長い会議テスト（詳細モード）
# ============================================
#
# 【期待される動作】
# START → summarize → check_length → extract_topics → extract_actions → format_doc → END
#

long_meeting = """
【プロジェクト進捗会議】2024年1月15日

田中: 今週の進捗を報告します。フロントエンドのUI改修は80%完了。
残りはモバイル対応です。来週水曜までに完了予定。

鈴木: バックエンドのAPI改修は予定より遅れています。
認証周りで問題が発生し、解決に2日かかりまし���。
来週金曜までには完了できる見込みです。

山田: QAテストの準備は完了しています。
来週月曜からテスト開始できます。
テスト項目は150件、3日間で実施予定です。

田中: 予算について確認です。追加で10万円必要になりそうです。
理由はサーバー増強のため。

鈴木: 承認します。経理に連絡しておきます。

次回の会議は来週月曜10時から。
""" * 5  # 5回繰り返して長くする（2000文字以上にする）

print("=" * 60)
print("【長い会議テスト（詳細モード）】")
print("=" * 60)
print(f"テキスト長: {len(long_meeting)}文字（2000文字以上）")
print("期待される分岐: 詳細モード（議題・アクション抽出を実行）")
print("")
print("-" * 60)
print("処理開始...")
print("-" * 60)

try:
    result = graph.invoke({
        "raw_text": long_meeting,
        "logs": []
    })
    
    print("")
    print("=" * 60)
    print("【生成された議事録】")
    print("=" * 60)
    print(result["final_doc"])
    
    print("=" * 60)
    print("【処理ログ】")
    print("=" * 60)
    for log in result["logs"]:
        print(f"  {log}")
    
    print("")
    print("✅ テスト成功！")
    
except Exception as e:
    print(f"❌ 実行エラー: {e}")
    import traceback
    traceback.print_exc()

---

# 検証結果まとめ

## チェックリスト

| 項目 | 状態 | 説明 |
|------|------|------|
| 構文エラー | ✅ | 全コードが正常に定義できた |
| インポート | ✅ | langgraph, typing が正しくインポートできた |
| State定義 | ✅ | MeetingStateが正しく定義できた |
| Node関数 | ✅ | 5つのNode関数が正しい型で定義できた |
| ルーティング | ✅ | 条件分岐が正しく動作した |
| グラフ組み立て | ✅ | compile()が成功した |
| 実行テスト | ✅ | 短い/長い会議の両方で動作確認できた |

## 次のステップ

### モックモードから本番モードへ
Vertex AI APIが使える環境では：
1. `USE_MOCK = False` に変更
2. `gcloud auth application-default login` で認証
3. 再度実行

### 実践用.pyファイル
このノートブックのコードを `.py` ファイルに分割して、本番運用できます：
- `state.py` - State定義
- `nodes.py` - Node関数
- `routing.py` - ルーティング関数
- `main.py` - グラフ組み立てと実行

---

お疲れ様でした！🎉